# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 10:50AM,261630,10,MSP221051,"Methapharm, Inc.",Released
1,Apr 24 2023 10:46AM,261629,15,30015914,"Alliance Pharma, Inc.",Released
2,Apr 24 2023 10:46AM,261629,15,30015915,"Alliance Pharma, Inc.",Released
3,Apr 24 2023 10:46AM,261629,15,30015916,"Alliance Pharma, Inc.",Released
4,Apr 24 2023 10:46AM,261629,15,30015917,"Alliance Pharma, Inc.",Released
5,Apr 24 2023 10:46AM,261629,15,30015918,"Alliance Pharma, Inc.",Released
6,Apr 24 2023 10:46AM,261629,15,30015919,"Alliance Pharma, Inc.",Released
7,Apr 24 2023 10:46AM,261629,15,30015921,"Alliance Pharma, Inc.",Released
8,Apr 24 2023 10:46AM,261629,15,30015924,"Alliance Pharma, Inc.",Released
9,Apr 24 2023 10:46AM,261629,15,30015926,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
11,261626,Released,2
12,261627,Released,18
13,261628,Released,1
14,261629,Released,22
15,261630,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261626,NaN,2.0
261627,NaN,18.0
261628,NaN,1.0
261629,NaN,22.0
261630,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261602,0.0,1.0
261612,0.0,29.0
261615,18.0,2.0
261616,0.0,1.0
261618,0.0,28.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261602,0,1
261612,0,29
261615,18,2
261616,0,1
261618,0,28


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261602,0,1
1,261612,0,29
2,261615,18,2
3,261616,0,1
4,261618,0,28


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261602,,1
1,261612,,29
2,261615,18,2
3,261616,,1
4,261618,,28


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc."
1,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc."
23,Apr 24 2023 10:43AM,261627,10,Bio-PRF
41,Apr 24 2023 10:41AM,261628,10,Methapharm-G
42,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC"
44,Apr 24 2023 10:26AM,261625,10,"CLINUVEL, Inc."
46,Apr 24 2023 10:24AM,261619,18,Seqirus
47,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC
97,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc."
98,Apr 24 2023 10:06AM,261620,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",,1
1,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",,22
2,Apr 24 2023 10:43AM,261627,10,Bio-PRF,,18
3,Apr 24 2023 10:41AM,261628,10,Methapharm-G,,1
4,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",,2
5,Apr 24 2023 10:26AM,261625,10,"CLINUVEL, Inc.",,2
6,Apr 24 2023 10:24AM,261619,18,Seqirus,,1
7,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,,50
8,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",,1
9,Apr 24 2023 10:06AM,261620,15,"Brookfield Pharmaceuticals, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,
1,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",22,
2,Apr 24 2023 10:43AM,261627,10,Bio-PRF,18,
3,Apr 24 2023 10:41AM,261628,10,Methapharm-G,1,
4,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",2,
5,Apr 24 2023 10:26AM,261625,10,"CLINUVEL, Inc.",2,
6,Apr 24 2023 10:24AM,261619,18,Seqirus,1,
7,Apr 24 2023 10:17AM,261622,20,Reliable 1 Laboratories LLC,50,
8,Apr 24 2023 10:06AM,261621,20,"Granules Pharmaceuticals, Inc.",1,
9,Apr 24 2023 10:06AM,261620,15,"Brookfield Pharmaceuticals, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,
1,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",22,
2,Apr 24 2023 10:43AM,261627,10,Bio-PRF,18,
3,Apr 24 2023 10:41AM,261628,10,Methapharm-G,1,
4,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,NaN
1,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",22,NaN
2,Apr 24 2023 10:43AM,261627,10,Bio-PRF,18,NaN
3,Apr 24 2023 10:41AM,261628,10,Methapharm-G,1,NaN
4,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",2,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 24 2023 10:50AM,261630,10,"Methapharm, Inc.",1,0.0
1,Apr 24 2023 10:46AM,261629,15,"Alliance Pharma, Inc.",22,0.0
2,Apr 24 2023 10:43AM,261627,10,Bio-PRF,18,0.0
3,Apr 24 2023 10:41AM,261628,10,Methapharm-G,1,0.0
4,Apr 24 2023 10:32AM,261626,10,"ClearSpec, LLC",2,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1831364,54,0.0
15,1046482,54,18.0
16,261602,1,0.0
18,261619,1,0.0
20,523243,51,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1831364,54,0.0
1,15,1046482,54,18.0
2,16,261602,1,0.0
3,18,261619,1,0.0
4,20,523243,51,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,54,0.0
1,15,54,18.0
2,16,1,0.0
3,18,1,0.0
4,20,51,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,54.0
1,15,Released,54.0
2,16,Released,1.0
3,18,Released,1.0
4,20,Released,51.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,20
Status,,,,,
Executing,0.0,18.0,0.0,0.0,0.0
Released,54.0,54.0,1.0,1.0,51.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,20
0,Executing,0.0,18.0,0.0,0.0,0.0
1,Released,54.0,54.0,1.0,1.0,51.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,20
0,Executing,0.0,18.0,0.0,0.0,0.0
1,Released,54.0,54.0,1.0,1.0,51.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()